# quake 3


## file management



### managing zips/pk3s



#### find pk3 files in zips?


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')

function readZip(zipFile, cb) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        // Take a look at the files
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        var pk3s = Object.values(zip.entries())
            .filter(entry => entry.name.includes('.pk3'))
        
        return importer.runAllPromises(pk3s.map((pk3, i) => new Promise((resolve) => {
            var pk3Path = '/Applications/ioquake3/bestmaps/' + path.basename(pk3.name);
            if(!fs.existsSync(pk3Path)) {
                zip.extract(pk3.name, pk3Path, err => {
                    if(err) {
                        console.log('Extract error ' + err)
                        return resolve()
                    }
                    console.log('Extracted ' + pk3.name);
                    resolve(pk3Path)
                });
            }
            resolve()
        }))).then(() => {
            zip.close()
            cb()
        })
    });
    
    zip.on('error', err => { cb() });
}

function extractAll(root) {
    var results = listInProject(root || '/Users/briancullinan/planet_quake_resources', '**/*.zip')
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readZip(r, resolve)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}

module.exports = extractAll;


#### find only bsps in map paks?



In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')


function readPak(zipFile, cb) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        for (const entry of Object.values(zip.entries())) {
            if(entry.name.includes('.bsp')) {
                console.log(entry.name)
                /*var pk3Path = '/Applications/ioquake3/bestmaps/' + path.basename(pk3.name);
                if(!fs.existsSync(pk3Path)) {
                    zip.extract(pk3.name, pk3Path, err => {
                        console.log((err ? 'Extract error ' : 'Extracted ') + pk3.name);
                        zip.close();
                        cb(zipFile);
                    });
                    return;
                }*/
            }
            if(entry.name.includes('levelshots/') && entry.name !== 'levelshots/') {
                var levelPath = '/Applications/ioquake3/bestmaps/levelshots/' + path.basename(entry.name)
                if(!fs.existsSync(levelPath)) {
                    zip.extract(entry.name, levelPath, err => {
                        console.log((err ? 'Extract error ' : 'Extracted ') + entry.name);
                        zip.close();
                        cb(zipFile);
                    });
                    return;
                }
            }
        }
        zip.close()
        cb();
    });
    
    zip.on('error', err => { cb() });
}


function extractPaks() {
    var results = listInProject('/Applications/ioquake3/bestmaps', '**/*.pk3')
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readPak(r, resolve)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}


module.exports = extractPaks;


#### list bsps in a pak?

For that annoying problem when you download a map pk3 and it's a different name than the map.


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');

function listBsps(zipFile) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        for (const entry of Object.values(zip.entries())) {
            if(entry.name.includes('.bsp')) {
                console.log(entry.name)
            }
        }
    });
    
    zip.on('error', err => {  });
}

module.exports = listBsps;


#### unpack entire folder of pk3s?



In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')
var mkdirpSync = importer.import('mkdirp')

function readPak(zipFile, cb, outdir) {
    if(!outdir) {
        outdir = '/Users/briancullinan/.q3a/unpacked/' + path.basename(zipFile) + 'dir'
    }
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        var index = Object.values(zip.entries())
        return Promise.all(index.map(entry => {
            var levelPath = path.join(outdir, entry.name)
            mkdirpSync(path.dirname(levelPath))
            return new Promise(resolve => {
                zip.extract(entry.name, levelPath, err => {
                    console.log((err ? 'Extract error ' : 'Extracted ') + entry.name);
                    resolve(zipFile);
                });
            })
        })).then(() => {
            zip.close();
            cb()
        })
    });
    
    zip.on('error', err => { cb() });
}


function extractPaks(inpaks, outdir) {
    var results = listInProject(path.dirname(inpaks), '**/' + path.basename(inpaks))
    results.sort()
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readPak(r, resolve, outdir)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}


module.exports = extractPaks;


### TODO: organize textures, find overrides (accidental and intentional)

Does TrenchBroom really require everything be in 1 folder?


TODO: all world, object and player textures run through waifu2x and SFTGAN


## convert quake 2 map to quake 3


#### the code

convert quake 2 map to quake 3?


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var replaceClasses = importer.import('replace known classes in map')
var replaceEntities = importer.import('replace entities in map')
var replaceTextures = importer.import('replace common textures in map')
var scaleMap = importer.import('scale quake map')
var addSkybox = importer.import('add skybox to map')

function convertMap(fileName) {
    var file;
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
    }
    
    file = replaceEntities(file)
    
    file = replaceClasses(file)
    
    // replace water with foggy water
    
    // replace message with target print
    
    // replace duplicate textures / don't interfere with basepack
    file = replaceTextures(file)
    
    // replace barrels with rocket launchers
    
    // replace animations and generate proper shaders
    // scale map 1.2x
    file = scaleMap(file, 1.15)
    
    file = addSkybox(file)
    
    file = addHints(file)
    
    // scale lighting with q3map2 bsp options
    var newFile = fileName.substr(0, fileName.length - path.extname(fileName).length)
    console.log(`writing ${newFile}`)
    fs.writeFileSync(newFile, file)
}

module.exports = convertMap;


#### scale quake map?



In [ ]:
var fs = require('fs')
var DIGITS = 100.0

function scaleMap(file, amount) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
    
    // replace all brushes with scaled values
    brushes.forEach(b => {
        var newBrush = b
        newBrush = newBrush.replace(/\(((\s*[0-9\.-]+\s*)*)\)/ig, (str, $1) => {
            return '( ' + $1.trim().split(/\s+/ig)
                .map(n => Math.round((n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
                .join(' ') + ' )'
        })
        // scale the texture on the brush
        newBrush = newBrush.replace(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+\s*){5,8})/ig, (str, $1, $2) => {
            return ') ' + $1 + ' ' + $2.trim().split(/\s+/ig)
                .map((n, i) => i >= 5 || i <= 2 ? n : (Math.round((n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) * amount * DIGITS) / DIGITS))
                .join(' ') + '\n'
        })
        file = file.replace(b, newBrush)
    })
    
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer.regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    origins.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map(n => Math.round((n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
            .join(' ')
        file = file.replace(new RegExp('"origin"\\s+"' + $1 + '"', 'ig'), '"origin" "' + newOrigin + '"')
    })
    var lips = importer.regexToArray(/"lip"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    lips.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map(n => Math.round((n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
            .join(' ')
        file = file.replace(new RegExp('"lip"\\s+"' + $1 + '"', 'ig'), '"lip" "' + newOrigin + '"')
    })
    
    // scale models
    var models = importer.regexToArray(/\{[\s\S^\}\{]*?"classname"\s+"misc_model"[\s\S^\}\{]*?\}/ig, file)
    models.forEach(m => {
        var newModel = m
        if(m.match(/"modelscale"/i)) {
            var match = (/"modelscale"\s+"([^"]*?)"/ig).exec(m)
            var scaled = (match[1].includes('.')
                ? parseFloat(match[1])
                : parseInt(match[1])) * amount
            newModel = newModel.replace(match[0], '"modelscale" "' + scaled + '"')
        } else {
            newModel = newModel.replace(/"classname"\s+"misc_model"/ig,
                                        '"classname" "misc_model"\n "modelscale" "' + amount + '"')
        }
        file = file.replace(m, newModel)
    })
    
    return file
}

module.exports = scaleMap


#### replace common textures in map?


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')

var common = [
  'areaportal',    'botclip',
  'caulk',         'clip',
  'clusterportal', 'cushion',
  'donotenter',    'full_clip',
  'hint',          'ladderclip',
  'ladderclip',    'lightgrid',
  'metalclip',     'missileclip',
  'nodraw',        'nodrawnonsolid',
  'nodrop',        'nolightmap',
  'origin',        'qer_mirror',
  'qer_portal',    'slick',
  'terrain',       'trigger',
  'weapclip',      'white'
];
var textures = {
    
}
function replaceTextures(file) {
    var used = []
    
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // get a list of common textures
    if(!common) {
        var files = fs.readdirSync('/Users/briancullinan/planet_quake_data/quake3-baseq3/common-spog.pk3dir/textures/common')
        common = files.map(f => f.substr(0, f.length - path.extname(f).length));
    }
    
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
    
    // replace all brushes with textures from common or berserker overrides
    brushes.forEach(b => {
        var newBrush = b
        Object.keys(textures).forEach(k => {
            newBrush = newBrush.replace(new RegExp('\\) ([^\\/\\)\\(]*?\\/)*?' + k, 'ig'),
                                        (str, $1) => (') ' + textures[k]))
        })
        common.forEach(k => {
            newBrush = newBrush.replace(new RegExp('\\) ([^\\/\\)\\(]*?\\/)*?' + k, 'ig'),
                                (str, $1) => (') common/' + (k === 'clip' ? 'hint' : k)))
        })
        // fix water, don't mix textures because
        //   trenchbroom rearranges verteces so water isn't always applied
        if(newBrush.includes('wter')) {
            newBrush = newBrush
                .replace(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm,
                         (str, $1, $2) => (') e1u1/bluwter ' + $2))
        }
        var texts = importer.regexToArray(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm, b, 1)
        texts.forEach($1 => {
            if(used.indexOf(path.dirname($1)) === -1) used.push(path.dirname($1))
        })
        
        file = file.replace(b, newBrush)
    })
    
    used = used.map(u => 'textures/' + u.replace('^\/|\/$|^\s*|\s*$', ''))
    file = file.replace(/"classname" "worldspawn"/ig,`"classname" "worldspawn"
"_tb_textures" "textures/common;${used.join(';')}"`)

    return file
}

module.exports = replaceTextures


#### replace known classes in map?



In [ ]:
var fs = require('fs')

var classes = {
    weapon_chaingun:                'weapon_lightning',
    misc_teleporter:                'trigger_teleport',
    weapon_hyperblasters:           'weapon_plasmagun',
    weapon_supershotgun:            'weapon_shotgun',
    info_teleport_destination:      'target_position',
    item_armor1:                    'item_armor_combat',
    item_armor2:                    'item_armor_combat',
    item_armorInv:                  'item_armor_body',
    item_artifact_envirosuit:       'item_enviro',
    item_artifact_invisibility:     'item_invis',
    item_artifact_super_damage:     'item_quad',
    item_cells:                     'ammo_lightning',
    item_rockets:                   'ammo_rockets',
    item_shells:                    'ammo_shells',
    item_spikes:                    'ammo_cells',
    weapon_nailgun:                 'weapon_plasmagun',
    weapon_supernailgun:            'weapon_plasmagun',
    weapon_supershotgun:            'weapon_shotgun',
    // replace target_help with target_print
    target_help:                    'target_print',
    func_door_rotating:             'func_rotating'
}
function replaceClasses(file) {
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // replace explosions with misc_model
    file = file.replace(/"classname" "target_explosion"/ig,
                        '"classname" "misc_model"\n "model" "models/objects/r_explode/tris.md2"')
    // TODO: replace explosions with animated delay
    // replace tigger_once with multiple wait -1
    file = file.replace(/"classname" "trigger_once"/ig,
                        '"classname" "trigger_multiple"\n "wait" "-1"')
    // replace classes
    Object.keys(classes).forEach(k => {
        file = file.replace(new RegExp('"classname" "' + k + '"', 'ig'),
                            '"classname" "' + classes[k] + '"')
    })
    return file
}

module.exports = replaceClasses


#### replace entities in map?



In [ ]:
var fs = require('fs')
var importer = require('../Core')

var MATCH_MODELS = /QUAKED\s+([^\s]*)\s+[\s\S]*?model="([^"]*)"/i;

var ents;
function replaceEntities(file, entities) {
    if(!entities) {
        entities = '/Users/briancullinan/planet_quake_data/quake2-baseq2/scripts/entities.def'
    }
    
    // parse entities definition
    if(!ents) {
        var entityStr = fs.readFileSync(entities).toString('utf-8')
        var matches = importer.regexToArray(/QUAKED\s+.*/ig, entityStr)
        ents = matches.reduce((obj, m, i, arr) => {
            var start = entityStr.indexOf(m)
            var end = entityStr.indexOf(arr[i + 1])
            if(end === -1) end = entityStr.length
            var groups = MATCH_MODELS.exec(entityStr.substring(start, end))
            if(!groups) return obj
            obj[groups[1]] = groups[2]
            return obj
        }, {})
    }
    
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // replace entities
    file = file.replace(/"misc_([^"]*)"/ig, ($0) => `"misc_model"\n "model" ${$0}`)
    
    // replace misc_ with models from ent
    Object.keys(ents).forEach(k => {
        if(!k.includes('misc_')) return
        file = file.replace(new RegExp('"' + k + '"', 'ig'), '"' + ents[k] + '"')
    })
    return file
}

module.exports = replaceEntities;


#### add a skybox and remove sky entities?

add skybox to map?

Because quake 2 maps leak somewhere during this process.


In [ ]:
var importer = require('../Core')
var {doIntersect} = importer.import('brush to vertex')

function getValues(file) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
   
    brushes = brushes.map(b => {
        var intersect = 0
        var brush = importer
            .regexToArray(/\(((\s*[0-9\.-]+\s*)*)\)/ig, b, 1)
            .map(m => m.trim().split(/\s+/ig)
                .map(n => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim()))))
        
        return [[
            Math.min.apply(null, brush.map(b => b[0])),
            Math.min.apply(null, brush.map(b => b[1])),
            Math.min.apply(null, brush.map(b => b[2]))
        ], [
            Math.max.apply(null, brush.map(b => b[0])),
            Math.max.apply(null, brush.map(b => b[1])),
            Math.max.apply(null, brush.map(b => b[2]))
        ]]
    })
 
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer
        .regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
        .map(o => o.trim().split(/\s+/ig)
            .map(n => (n.includes('.')
                ? parseFloat(n.trim())
                : parseInt(n.trim()))))
    
    origins = origins.concat.apply(origins, brushes)
        .filter(o => o && isFinite(o[0]))
    
    return [[
        Math.min.apply(null, origins.map(b => b[0])),
        Math.min.apply(null, origins.map(b => b[1])),
        Math.min.apply(null, origins.map(b => b[2]))
    ], [
        Math.max.apply(null, origins.map(b => b[0])),
        Math.max.apply(null, origins.map(b => b[1])),
        Math.max.apply(null, origins.map(b => b[2]))
    ]]
}


function addSkybox(fileName) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
    } else {
        file = fileName
    }

    var brushes = importer.regexToArray(/\{[^\{}]*?\}\s*/ig, file)
        .filter(b => {
            if(b.includes('/sky')) {
                file = file.replace(b, '')
                return false
            }
            return true
        })
    
    var vs = getValues(file)
    
    // TODO: use a fancy for loop instead contains each corner and extends towards the next two points?
    var points = [
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
        [vs[0][0]-16, vs[0][1], vs[0][2]],
        [vs[0][0],    vs[1][1], vs[1][2]],
        
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
        
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
    ]
    var newBrush = `
{ // brush 0
( ${vs[0][0]} 65535 65535 ) ( ${vs[0][0]} -65535 65535 ) ( ${vs[0][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( ${vs[1][0]} -65535 65535 ) ( ${vs[1][0]} 65535 65535 ) ( ${vs[1][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( -65535 ${vs[0][1]} 65535 ) ( 65535 ${vs[0][1]} 65535 ) ( -65535 ${vs[0][1]} -65535 )  e1u1/sky 0 0 0 1 1 0 0 0
( 65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 -65535 ${vs[0][2]-16} ) ( 65535 65535 ${vs[0][2]-16} ) ( -65535 -65535 ${vs[0][2]-16} ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 65535 ${vs[0][2]} ) ( 65535 -65535 ${vs[0][2]} ) ( -65535 -65535 ${vs[0][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
}

{ // brush 0
( ${vs[0][0]-16} 65535 65535 ) ( ${vs[0][0]-16} -65535 65535 ) ( ${vs[0][0]-16} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( ${vs[0][0]} -65535 65535 ) ( ${vs[0][0]} 65535 65535 ) ( ${vs[0][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( -65535 ${vs[0][1]} 65535 ) ( 65535 ${vs[0][1]} 65535 ) ( -65535 ${vs[0][1]} -65535 )  e1u1/sky 0 0 0 1 1 0 0 0
( 65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 -65535 ${vs[0][2]} ) ( 65535 65535 ${vs[0][2]} ) ( -65535 -65535 ${vs[0][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 65535 ${vs[1][2]} ) ( 65535 -65535 ${vs[1][2]} ) ( -65535 -65535 ${vs[1][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
}

{ // brush 0
( ${vs[0][0]} 65535 65535 ) ( ${vs[0][0]} -65535 65535 ) ( ${vs[0][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( ${vs[1][0]} -65535 65535 ) ( ${vs[1][0]} 65535 65535 ) ( ${vs[1][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( -65535 ${vs[0][1]-16} 65535 ) ( 65535 ${vs[0][1]-16} 65535 ) ( -65535 ${vs[0][1]-16} -65535 )  e1u1/sky 0 0 0 1 1 0 0 0
( 65535 ${vs[0][1]} 65535 ) ( -65535 ${vs[0][1]} 65535 ) ( -65535 ${vs[0][1]} -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 -65535 ${vs[0][2]} ) ( 65535 65535 ${vs[0][2]} ) ( -65535 -65535 ${vs[0][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 65535 ${vs[1][2]} ) ( 65535 -65535 ${vs[1][2]} ) ( -65535 -65535 ${vs[1][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
}

{ // brush 0
( ${vs[0][0]} 65535 65535 ) ( ${vs[0][0]} -65535 65535 ) ( ${vs[0][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( ${vs[1][0]} -65535 65535 ) ( ${vs[1][0]} 65535 65535 ) ( ${vs[1][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( -65535 ${vs[0][1]} 65535 ) ( 65535 ${vs[0][1]} 65535 ) ( -65535 ${vs[0][1]} -65535 )  e1u1/sky 0 0 0 1 1 0 0 0
( 65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 -65535 ${vs[1][2]} ) ( 65535 65535 ${vs[1][2]} ) ( -65535 -65535 ${vs[1][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 65535 ${vs[1][2]+16} ) ( 65535 -65535 ${vs[1][2]+16} ) ( -65535 -65535 ${vs[1][2]+16} ) e1u1/sky 0 0 0 1 1 0 0 0
}

{ // brush 0
( ${vs[1][0]} 65535 65535 ) ( ${vs[1][0]} -65535 65535 ) ( ${vs[1][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( ${vs[1][0]+16} -65535 65535 ) ( ${vs[1][0]+16} 65535 65535 ) ( ${vs[1][0]+16} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( -65535 ${vs[0][1]} 65535 ) ( 65535 ${vs[0][1]} 65535 ) ( -65535 ${vs[0][1]} -65535 )  e1u1/sky 0 0 0 1 1 0 0 0
( 65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 -65535 ${vs[0][2]} ) ( 65535 65535 ${vs[0][2]} ) ( -65535 -65535 ${vs[0][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 65535 ${vs[1][2]} ) ( 65535 -65535 ${vs[1][2]} ) ( -65535 -65535 ${vs[1][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
}

{ // brush 0
( ${vs[0][0]} 65535 65535 ) ( ${vs[0][0]} -65535 65535 ) ( ${vs[0][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( ${vs[1][0]} -65535 65535 ) ( ${vs[1][0]} 65535 65535 ) ( ${vs[1][0]} 65535 -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( -65535 ${vs[1][1]} 65535 ) ( 65535 ${vs[1][1]} 65535 ) ( -65535 ${vs[1][1]} -65535 )  e1u1/sky 0 0 0 1 1 0 0 0
( 65535 ${vs[1][1]+16} 65535 ) ( -65535 ${vs[1][1]+16} 65535 ) ( -65535 ${vs[1][1]+16} -65535 ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 -65535 ${vs[0][2]} ) ( 65535 65535 ${vs[0][2]} ) ( -65535 -65535 ${vs[0][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
( 65535 65535 ${vs[1][2]} ) ( 65535 -65535 ${vs[1][2]} ) ( -65535 -65535 ${vs[1][2]} ) e1u1/sky 0 0 0 1 1 0 0 0
}
`

    var exp = (/\{*\s*\/\/\s*brush\s*0\s*\{*/ig)
    var match = exp.exec(file)
    var pos = exp.lastIndex
    file = file.substr(0, pos - match[0].length) + newBrush + file.substr(pos - match[0].length)
    
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        console.log(`writing ${fileName}`)
        fs.writeFileSync(fileName, file)
    } else {
        return file
    }
}

module.exports = addSkybox



#### draw hints from x and y coordinates?

draw hints in map?



In [ ]:
var MAPS_HINTS = {
    'base1-a': [
        [-648, -209],
        [-648, 235],
        [-400, 235],
        [-400, 514],
        [300, 514],
        [300, 1922],
        [9999, 1922],
        [9999, -9999],
        [-648, -9999],
        [-648, -209]
    ],
    'base1-b': [
        [-648, -209],
        [-648, 235],
        [-400, 235],
        [-400, 514],
        [300, 514],
        [300, 1922],
        [-1300, 2174],
        [-1300, 648],
        [-648, -209],
    ],
    'base1-c': [
        [-1300, 1000],
        [-1300, 2174],
        [-9999, 9999],
        [-9999, 1000],
        [-1300, 1000],
    ]
}

function addHints(file, hints) {
    var zMin = 0, zMax = 0
    
    
    return file
}

module.exports = {
    MAPS_HINTS,
    addHints
}


#### TODO: convert quake 2 entities to quake 3?





```
Quake 1:
------------------------------------------------
func_plat                      *
func_door                      *
info_null                      *
info_player_deathmatch         *
item_health                    *
info_teleport_destination      target_position
item_armor1                    item_armor_combat
item_armor2                    item_armor_combat
item_armorInv                  item_armor_body
item_artifact_envirosuit       item_enviro
item_artifact_invisibility     item_invis
item_artifact_super_damage     item_quad
item_cells                     ammo_lightning
item_rockets                   ammo_rockets
item_shells                    ammo_shells
item_spikes                    ammo_cells
weapon_nailgun                 weapon_plasmagun
weapon_supernailgun            weapon_plasmagun
weapon_supershotgun            weapon_shotgun
weapon_rocketlauncher          *
light                          *
trigger_hurt                   *
trigger_multiple               *
trigger_push                   *
trigger_teleport               *
weapon_grenadelauncher         *
weapon_lightning               *

Quake 2:
-------------------------------------------------
ammo_bullets               *
ammo_cells                 *
ammo_grenades              *
ammo_rockets               *
ammo_shells                *
ammo_slugs                 *
func_button                *
func_door                  *
func_group                 *
func_plat                  *
func_rotating              *
func_timer                 *
func_train                 *
info_notnull               *
info_null                  *
info_player_deathmatch     *
info_player_intermission   *
item_armor_body            *
item_armor_combat          *
item_armor_shard           *
item_enviro                *
item_health                *
item_health_large          *
item_health_mega           *
item_health_small          *
item_quad                  *
light                      *
misc_teleporter_dest       *
path_corner                *
target_laser               *
target_speaker             *
trigger_always             *
trigger_hurt               *
trigger_multiple           *
trigger_push               *
weapon_bfg                 *
weapon_chaingun            weapon_lightning
misc_teleporter            trigger_teleport 
weapon_hyperblasters       weapon_plasmagun
weapon_supershotgun        weapon_shotgun
weapon_grenadelauncher     *
weapon_machinegun          *
weapon_railgun             *
weapon_rocketlauncher      *
weapon_shotgun             *
```


### misc map/bsp scripts



#### get vertex from brush strokes?

brush to vertex?


In [ ]:
  
// Given three colinear points p, q, r, the function checks if 
// point q lies on line segment 'pr' 
function onSegment(p, q, r) 
{ 
    if (q.x <= Math.max(p.x, r.x) && q.x >= Math.min(p.x, r.x) && 
            q.y <= Math.max(p.y, r.y) && q.y >= Math.min(p.y, r.y)) {
        
        var xinters = (q.y - p.y) * (r.x - p.x) / (r.y - p.y) + p.x;
        return xinters; 
    }
    return false; 
} 
  
// To find orientation of ordered triplet (p, q, r). 
// The function returns following values 
// 0 --> p, q and r are colinear 
// 1 --> Clockwise 
// 2 --> Counterclockwise 
function orientation(p, q, r) 
{ 
    var val = (q.y - p.y) * (r.x - q.x) - 
              (q.x - p.x) * (r.y - q.y); 
  
    if (val == 0) return 0;  // colinear 
    return (val > 0)? 1: 2; // clock or counterclock wise 
} 
  
// The function that returns true if line segment 'p1q1' 
// and 'p2q2' intersect. 
function doIntersect(p1, q1, p2, q2) 
{ 
    // Find the four orientations needed for general and 
    // special cases 
    var o1 = orientation(p1, q1, p2); 
    var o2 = orientation(p1, q1, q2); 
    var o3 = orientation(p2, q2, p1); 
    var o4 = orientation(p2, q2, q1); 
  
    // General case 
    if (o1 != o2 && o3 != o4) {
        if(o1 === 1) {
            var yOff = (q1.y - p1.y)
            if(yOff === 0) yOff = 1
            var xinters = (p2.y - p1.y) * (q1.x - p1.x) / yOff + p1.x;
            if(xinters === p2.y || xinters === p1.y) return false
            if(xinters > 9000 || xinters < -9000) {
                debugger
            }
            return [xinters, p1.y]
        } else {
            var yOff = (q2.y - p2.y)
            if(yOff === 0) yOff = 1
            var xinters = (p1.y - p2.y) * (q2.x - p2.x) / yOff + p2.x;
            if(xinters === p2.y || xinters === p1.y) return false
            if(xinters > 9000 || xinters < -9000) {
                debugger
            }
            return [xinters, p2.y]
        }
    }
  
    // Special Cases 
    // p1, q1 and p2 are colinear and p2 lies on segment p1q1 
    var xinters = 0;
    if (o1 == 0 && (xinters = onSegment(p1, p2, q1)) !== false) [xinters, p2.y]
  
    // p1, q1 and p2 are colinear and q2 lies on segment p1q1 
    if (o2 == 0 && (xinters = onSegment(p1, q2, q1)) !== false) [xinters, q2.y]
  
    // p2, q2 and p1 are colinear and p1 lies on segment p2q2 
    if (o3 == 0 && (xinters = onSegment(p2, p1, q2)) !== false) [xinters, p1.y]
  
     // p2, q2 and q1 are colinear and q1 lies on segment p2q2 
    if (o4 == 0 && (xinters = onSegment(p2, q1, q2)) !== false) [xinters, q1.y] 
  
    return false; // Doesn't fall in any of the above cases 
} 

/*
// Returns true if the point p lies inside the polygon[] with n vertices 
bool isInside(Point polygon[], int n, Point p) 
{ 
    // There must be at least 3 vertices in polygon[] 
    if (n < 3)  return false; 
  
    // Create a point for line segment from p to infinite 
    Point extreme = {INF, p.y}; 
  
    // Count intersections of the above line with sides of polygon 
    int count = 0, i = 0; 
    do
    { 
        int next = (i+1)%n; 
  
        // Check if the line segment from 'p' to 'extreme' intersects 
        // with the line segment from 'polygon[i]' to 'polygon[next]' 
        if (doIntersect(polygon[i], polygon[next], p, extreme)) 
        { 
            // If the point 'p' is colinear with line segment 'i-next', 
            // then check if it lies on segment. If it lies, return true, 
            // otherwise false 
            if (orientation(polygon[i], p, polygon[next]) == 0) 
               return onSegment(polygon[i], p, polygon[next]); 
  
            count++; 
        } 
        i = next; 
    } while (i != 0); 
  
    // Return true if count is odd, false otherwise 
    return count&1;  // Same as (count%2 == 1) 
} 
*/
module.exports = {
    doIntersect
}

#### copy brushes inside a polygon?




#### list noises in a quake 3 map?



In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')

function listNoise() {
    var contents = fs.readFileSync('/Users/briancullinan/.q3a/baseq3/baseq2.pk3dir/maps/base1.map').toString('utf8')
    var noises = importer.regexToArray(/noise.*?"\s*"(.*?)"/ig, contents, 1)
    return {all: noises, missing: noises.filter(n => {
        return !fs.existsSync(path.join('/Users/briancullinan/.q3a/baseq3/baseq2.pk3dir/sound/', n))
    })}
}

module.exports = listNoise